In [2]:
#imports
import requests
import pandas as pd
import os
import datetime as dt
from google.cloud import bigquery
from google.cloud import storage
from shapely.geometry  import shape, Point
from geopy.geocoders import Nominatim
import numpy as np
import meteostat as ms
import io
import json
from geopy import distance
import glob

In [3]:
CREDS = "C:/Users/tkkim/gcp_keys/capstone-team51-366963bafc54.json"
storage_client = storage.Client.from_service_account_json(json_credentials_path=CREDS,project='capstone-team51')
bq_client = bigquery.Client.from_service_account_json(json_credentials_path=CREDS,project='capstone-team51')
#client = bigquery.Client(project='capstone-team51')


In [4]:
bucket = storage_client.get_bucket('capstone-team51-data')

In [5]:
def load_df_gcs(prefix, filename):
    df = pd.DataFrame()
    for blob in storage_client.list_blobs('capstone-team51-data', prefix=prefix, delimiter='/'):
        #print(blob.name)
        if filename in blob.name:
            #print(blob.name)
            data = blob.download_as_bytes()
            smalldf = pd.read_csv(io.BytesIO(data))
            #print(smalldf)
            df = pd.concat([df, smalldf])
    return df

In [6]:
base_df = load_df_gcs('raw_crimes/', 'chunk')
three11_df = load_df_gcs('raw_311/', 'chunk')
env_df = load_df_gcs('raw_environmental/', 'chunk')
weather_df = load_df_gcs('raw_weather/', 'data')
offenders_df = load_df_gcs('supporting/off_commarea/', 'csv')

In [7]:
commarea = json.loads(bucket.blob('supporting/geojsons/Boundaries - Community Areas (current).geojson').download_as_string())
stations_df = load_df_gcs('supporting/', 'Police_Stations_20240120.csv')

In [8]:
def find_closest(lat, long):
    if lat == 0 or long == 0:
        return 'No Lat/Long', 999999
    else:
        try:
            input_latlong = (long,lat)
            #distance_list = []
            pdf = stations_df.copy()
            pdf['distance'] = pdf.apply(lambda x: distance.distance(input_latlong, (x.LONGITUDE,x.LATITUDE)).miles, axis=1)
            nearest_station = pdf.iloc[pdf['distance'].idxmin()]
            #print(nearest_station)
            return nearest_station['ADDRESS'], nearest_station['distance']
            """for row in POLICE_DF.iterrows():
                distance_holder = {}
                station_latlong = [row['LATITUDE'], row['LONGITUDE']]
                distance_holder['station'] = row['ADDRESS']
                distance_holder['distance'] = distance.distance(input_latlong, station_latlong).miles

                distance_list.append(distance_holder)
                distance_df = pd.DataFrame(distance_list)
                nearest_station = distance_df.iloc[distance_df['distance'].idxmin()]
                print(f"{distance_df.head()}")
                return nearest_station['station'], nearest_station['distance'] """
        except Exception as e:
            #print(e)
            return 'Error something something', 999999

In [9]:
def get_commarea_env(df_, commarea):
    dataframe = df_.copy()
    dataframe['community_area']=''
    for row in dataframe.index:
        point = Point(dataframe.loc[row,'longitude'],dataframe.loc[row,'latitude'])
        #print(point.xy)
        for feature in commarea['features']:
            polygon = shape(feature['geometry'])
            if polygon.contains(point):
                dataframe.loc[row, 'community_area'] = feature['properties']['area_numbe']
                
    return dataframe

In [16]:
data_path = './data/archive/'
cur_dir = os.getcwd()
os.chdir(data_path)
files = glob.glob("*.json")
os.chdir(cur_dir)

In [17]:
crimes_dict = {}
offenders_dict = {}
env_dict = {}
crimes_i = 0
env_i = 0
offenders_i = 0

for file in files:
    name = file.split(sep="_")[0]
    if name == 'crime5':
        crimes_dict[name+f'_{crimes_i}'] = pd.read_json((data_path+file), lines=True)
        crimes_i += 1
    elif name == 'offenders':
        continue
    elif name == 'envcomplaint5':
        env_dict[name+f'_{env_i}'] = pd.read_json((data_path+file), lines=True)
        env_i += 1
    else:
        print(f'error on {file}')

def merge_df(dictionary):
    merged_df = pd.DataFrame()
    for datfram in dictionary:
        merged_df= pd.concat([merged_df,dictionary[datfram]],axis=0)
        print(merged_df.shape[0])
    return merged_df

error on crime_chunk_0.json
error on envcomplaint_chunk_0.json


In [18]:
merged_crimes = merge_df(crimes_dict)
merged_envcomplaints = merge_df(env_dict)

500000
1000000
1181139
9410


In [25]:
merged_crimes[['closest_station', 'closest_station_distance']] = merged_crimes.apply(lambda x: find_closest(x.latitude, x.longitude), axis=1, result_type='expand')

In [26]:
# merge offenders in
big_df = merged_crimes.merge(offenders_df, how='left', on='community_area')
big_df['date'] = pd.to_datetime(big_df['date']).dt.date
big_df['community_area'] = big_df['community_area'].fillna(9999).astype('int64')

In [28]:
oldweather = pd.read_csv('./weather_data/weather_daily_data.csv')

In [29]:
merged_weather = pd.concat([oldweather, weather_df], axis=0)

In [36]:
merged_crimes.shape

(1181139, 26)

In [37]:
new_df.shape

(1181139, 40)

In [33]:
# merge weather in
merged_weather.rename(columns={'time':'date'}, inplace=True)
merged_weather['date'] = pd.to_datetime(merged_weather['date']).dt.date
big_df['date'] = pd.to_datetime(big_df['date']).dt.date
new_df = big_df.merge(merged_weather, how='left', on='date')

In [49]:
merged_weather.to_csv('./compiled_weather_daily.csv', index=False)

In [34]:
new_df.head()

,id,case_number,date,block,iucr,primary_type,description,location_description,arrest,domestic,...,prcp,snow,wdir,wspd,wpgt,pres,tsun,tempmax_f,tempmin_f,tempavg_f
0,13211437,JG425074,2019-01-01,009XX N HAMLIN AVE,1562,SEX OFFENSE,AGGRAVATED CRIMINAL SEXUAL ABUSE,RESIDENCE,False,False,...,0.0,0.0,342.0,14.2,NaN,1024.1,NaN,35.96,26.96,30.74
1,11895528,JC515003,2019-01-01,059XX W LELAND AVE,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,None,False,False,...,0.0,0.0,342.0,14.2,NaN,1024.1,NaN,35.96,26.96,30.74
2,12325061,JE180772,2019-01-01,045XX S CHAMPLAIN AVE,0265,CRIMINAL SEXUAL ASSAULT,AGGRAVATED - OTHER,RESIDENCE,False,False,...,0.0,0.0,342.0,14.2,NaN,1024.1,NaN,35.96,26.96,30.74
3,12298601,JE148751,2019-01-01,057XX S WOLCOTT AVE,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,RESIDENCE,False,False,...,0.0,0.0,342.0,14.2,NaN,1024.1,NaN,35.96,26.96,30.74
4,12312332,JE165297,2019-01-01,009XX W 76TH ST,0266,CRIMINAL SEXUAL ASSAULT,PREDATORY,RESIDENCE,False,False,...,0.0,0.0,342.0,14.2,NaN,1024.1,NaN,35.96,26.96,30.74


In [35]:
new_df.to_csv('oneoff_crimeoffendersweather.csv', index=False)

In [23]:
merged_crimes

'c:\\Users\\tkkim\\Desktop\\498_winter_group_bfmnt\\code_base\\python'

In [ ]:
get_commarea_env(merged_envcomplaints, commarea)

In [38]:
data_path = './311/'
cur_dir = os.getcwd()

os.chdir(data_path)
files = glob.glob("*.json")
os.chdir(cur_dir)

In [40]:
#311

three11holder = pd.DataFrame()
for file in files:
    three11holder = pd.concat([three11holder,pd.read_json((data_path+file), lines=True)],axis=0)

In [45]:
merged_311 = pd.concat([three11_df, three11holder], axis=0)

In [46]:
grouped_date = merged_311.groupby([merged_311['created_date'],'community_area','sr_type']).agg(total_count=('sr_number','count')).reset_index()
pivoted = grouped_date.pivot_table(index=['created_date','community_area'],columns='sr_type',values='total_count',aggfunc='sum').fillna(0).reset_index()
pivoted['created_date'] = pd.to_datetime(pivoted['created_date'])
pivoted.rename(columns={'created_date':'date'}, inplace=True)
pivoted['community_area'] = pivoted['community_area'].astype(int)
pivoted['date'] = pd.to_datetime(pivoted['date']).dt.date

In [47]:
pivoted.to_csv('oneoff_311.csv', index=False)
merged_311.to_csv('oneoff_311_all.csv', index=False)

In [48]:
pivoted.head()

sr_type,date,community_area,Abandoned Vehicle Complaint,Aircraft Noise Complaint,Alley Light Out Complaint,Alley Pothole Complaint,Alley Sewer Inspection Request,Animal In Trap Complaint,Bee/Wasp Removal,Bicycle Request/Complaint,...,Vicious Animal Complaint,Wage Complaint,Water Lead Test Kit Request,Water Lead Test Visit Request,Water On Street Complaint,Water Quality Concern,Water in Basement Complaint,Weed Removal Request,Wire Basket Request,Yard Waste Pick-Up Request
0,2019-01-01,53,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2019-01-01,19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2019-01-01,46,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2019-01-01,4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2019-01-01,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
